In [1]:
from numpy import loadtxt
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
tr_1 = pd.read_table("fin_shuf_c_nc", encoding='utf-8', engine='python')
tr_2 = pd.read_table("shuf_tot_train_lncFinder", encoding='utf-8', engine='python')
tr_3 = pd.read_table("shuf_total_train_human_mouse", encoding='utf-8', engine='python')

In [5]:
tot = tr_1.append(tr_2, ignore_index=True)
tot = tot.append(tr_3, ignore_index=True)
tot = tot.drop(['#ID'], axis = 1)

In [9]:
tot_model = pd.get_dummies(tot)

In [11]:
model = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [12]:
prot_feat = ['instability', 'gravy', 'Mw', 'pI', 'PW', 'peptide_length']
rna_feat = ['ORF_coverage', 'ORF_integrity', 'GC1', 'GC2', 'GC3']
dna_feat = ['transcript_length', 'stop_codon_num', 'Fickett_score']

label = ['#label_noncoding']



y_train = tot_model[label]

x_prot_train = tot[prot_feat]
x_rna_train = tot[rna_feat]
x_dna_train = tot[dna_feat]

final_feats = []

f_test = ['transcript_length', 'Fickett_score', 'ORF_coverage', 'ORF_integrity', 'GC1', 'GC2', 'GC3', 'stop_codon_num', 'instability', 'gravy', 'pI', 'Mw', 'PW']
l_test = ['#label_noncoding']

In [14]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
chi_2 = SelectKBest(chi2, k=2)
chi_dna = chi_2.fit(x_dna_train, y_train)
k = chi_dna.get_support()
#print(x_dna_train[1])
for i in range(0, len(k)):
    if(k[i]==True):
        final_feats.append(dna_feat[i])
print(final_feats)

['transcript_length', 'stop_codon_num']


In [15]:
from sklearn.feature_selection import RFE
#for protein, rfe
rfe = RFE(model, 3)
rfe_prot = rfe.fit(x_prot_train, y_train)
k = rfe_prot.get_support()
#print(x_dna_train[1])
for i in range(0, len(k)):
    if(k[i]==True):
        #print("i is %d"%i)
        #print("feature is " + x_dna_train[i])
        final_feats.append(prot_feat[i])
print(final_feats)

/home/thesis/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['transcript_length', 'stop_codon_num', 'gravy', 'Mw', 'pI']


In [16]:
#for rna_feats, rfe
#rfe = RFECV(model, min_features_to_select=3, cv=5)
rfe = RFE(model, 3)
rfe_rna = rfe.fit(x_rna_train, y_train)
k = rfe_rna.get_support()
#print(x_dna_train[1])
for i in range(0, len(k)):
    if(k[i]==True):
        #print("i is %d"%i)
        #print("feature is " + x_dna_train[i])
        final_feats.append(rna_feat[i])
print(final_feats)

['transcript_length', 'stop_codon_num', 'gravy', 'Mw', 'pI', 'GC1', 'GC2', 'GC3']


In [19]:
x_fin_feats_train = tot[final_feats]

In [20]:
#for the final feats, another rfe and select 5 best feats
fin_fin_feats = []
#rfe = RFECV(model, min_features_to_select=4, cv=5)
rfe = RFE(model, 4)
rfe_rna = rfe.fit(x_fin_feats_train, y_train)
k = rfe_rna.get_support()
#print(x_dna_train[1])
for i in range(0, len(k)):
    if(k[i]==True):
        #print("i is %d"%i)
        #print("feature is " + x_dna_train[i])
        fin_fin_feats.append(final_feats[i])


fin_fin_feats.append('Fickett_score')
#fin_fin_feats.append('ORF_coverage')
print fin_fin_feats

['gravy', 'Mw', 'pI', 'GC2', 'Fickett_score']


In [ ]:
tr_1 = ['instability', 'gravy', 'pI', 'ORF_coverage', 'Fickett_score']
tr_2 = ['gravy', 'Mw', 'GC1', 'GC2', 'Fickett_score']
tr_3 = ['instability', 'Mw', 'pI', 'GC2', 'Fickett_score']